In [1]:
import sys
print(sys.version)

3.11.5 (main, Sep 11 2023, 13:54:46) [GCC 11.2.0]


In [2]:
import torch
print(torch.__version__)

2.1.1+cu118


In [3]:
from torch_geometric.data import Data
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GATConv
import torch.nn.functional as F
import torch.nn as nn

In [4]:
from tqdm import tqdm
import numpy as np

In [5]:
device = torch.device("cuda")

In [6]:
cora_dataset = Planetoid('/tmp/cora', 'cora')

Processing...
Done!


In [8]:
cora_data = cora_dataset[0]

In [9]:
# For debug use only
num_nodes = cora_data.num_nodes
print('cora has {} nodes'.format(num_nodes))

num_edges = cora_data.num_edges
print('cora has {} edges'.format(num_edges))

cora has 2708 nodes
cora has 10556 edges


In [10]:
# For debug use only
print(cora_data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [24]:
# For debug use only
cora_x_train = cora_data.x[cora_data.train_mask]
cora_x_val = cora_data.x[cora_data.val_mask]
cora_x_test = cora_data.x[cora_data.test_mask]

print("number of nodes in cora train set,", cora_x_train.shape[0])
print("number of nodes in cora val set,", cora_x_val.shape[0])
print("number of nodes in cora test set,", cora_x_test.shape[0])

number of nodes in cora train set, 140
number of nodes in cora val set, 500
number of nodes in cora test set, 1000


In [12]:
# For debug use only
print(cora_data.y)
print(cora_data.y.shape)
s = set()
histogram = np.zeros(7)
for label in cora_data.y:
    s.add(label.item())
    histogram[label.item()]+=1
print(s)
print(histogram)

tensor([3, 4, 4,  ..., 3, 3, 3])
torch.Size([2708])
{0, 1, 2, 3, 4, 5, 6}
[351. 217. 418. 818. 426. 298. 180.]


In [13]:
# For debug use only
print(cora_data.x.shape)
print(cora_data.x[170:180])
print(cora_data.num_features)
print(cora_data.num_nodes)
print(cora_data.num_node_types)
print(type(cora_data))

torch.Size([2708, 1433])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
1433
2708
1
<class 'torch_geometric.data.data.Data'>


In [25]:
# Define the GAT model
class GAT(torch.nn.Module):
    # hidden channels will be the embedding dimension for each attention head
    # after applying the first GAT layer.
    def __init__(self, in_channels, hidden_channels, 
                 num_heads, dropout_rate, num_classes):
        super().__init__()
        
        self.dropout_rate = dropout_rate
        
        self.hidden_channels = hidden_channels
        self.num_heads = num_heads
        
        self.conv1 = GATConv(in_channels, hidden_channels, heads=num_heads, 
                                dropout=dropout_rate)
        self.conv2 = GATConv(hidden_channels*num_heads, num_classes, 
                                dropout=dropout_rate, concat=False)

    def forward(self, x, edge_index):
        out = F.dropout(x, p=self.dropout_rate, training=self.training)
        
        out = self.conv1(out, edge_index)
        assert out.shape[-1] == self.hidden_channels * self.num_heads
        
        out = F.elu(out)
        out = F.dropout(out, p=self.dropout_rate, training=self.training)
        
        out = self.conv2(out, edge_index)
        return out

In [26]:
num_heads = 8
dropout_rate = 0.4
emb_dim1 = 8
lr = 0.005

cora_num_classes = len(cora_data.y.unique())
assert cora_num_classes == 7

cora_model = GAT(cora_data.num_features, emb_dim1, num_heads, dropout_rate, 
            cora_num_classes).to(device)
cora_data_gpu = cora_data.to(device)
optimizer = torch.optim.Adam(cora_model.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss()

In [27]:
def train(model, data, optimizer, loss_fn):
    model.train()
    optimizer.zero_grad()
    
    pred = model(data.x, data.edge_index)
    loss = loss_fn(pred[data.train_mask], data.y[data.train_mask])
    
    loss.backward()
    optimizer.step()
    
    return loss

In [28]:
@torch.no_grad()
def evaluate(model, data, test_mask):
    accuracy_list = [0.0, 0.0]
    loss_list = [0.0, 0.0]
    model.eval()

    logits = model(data.x, data.edge_index)
    pred = logits.argmax(dim=-1)
    
    for i, mask in enumerate([data.train_mask, test_mask]):
        accuracy_list[i] = pred[mask].eq(data.y[mask]).float().mean().item()
        loss_list[i] = loss_fn(logits[mask], data.y[mask]).item()

    return accuracy_list, loss_list

In [38]:
num_epochs = 100
log_freq = 10

for epoch in tqdm(range(num_epochs), desc="Training Epochs"):
    loss = train(cora_model, cora_data, optimizer, loss_fn)
    acc_list, loss_list = evaluate(cora_model, cora_data, cora_data.val_mask)
    
    if (epoch % log_freq == 0) or (epoch + 1 == num_epochs):
        print(f"Epoch: {epoch+1}, Loss: {loss:.4f}")
        print(f"           Train Acc: {acc_list[0]:.4f}, Train Loss: {loss_list[0]:.4f}, Val Acc: {acc_list[1]:.4f}, Val Loss: {loss_list[1]:.4f}")

Training Epochs:  33%|███▎      | 33/100 [00:00<00:00, 168.66it/s]

Epoch: 1, Loss: 0.0546
           Train Acc: 1.0000, Train Loss: 0.0000, Val Acc: 0.7700, Val Loss: 1.7754
Epoch: 11, Loss: 0.0752
           Train Acc: 1.0000, Train Loss: 0.0000, Val Acc: 0.7660, Val Loss: 1.7682
Epoch: 21, Loss: 0.1474
           Train Acc: 1.0000, Train Loss: 0.0000, Val Acc: 0.7680, Val Loss: 1.7280
Epoch: 31, Loss: 0.1045
           Train Acc: 1.0000, Train Loss: 0.0000, Val Acc: 0.7640, Val Loss: 1.7167


Training Epochs:  74%|███████▍  | 74/100 [00:00<00:00, 189.71it/s]

Epoch: 41, Loss: 0.1554
           Train Acc: 1.0000, Train Loss: 0.0000, Val Acc: 0.7600, Val Loss: 1.7165
Epoch: 51, Loss: 0.0976
           Train Acc: 1.0000, Train Loss: 0.0000, Val Acc: 0.7760, Val Loss: 1.6881
Epoch: 61, Loss: 0.1134
           Train Acc: 1.0000, Train Loss: 0.0000, Val Acc: 0.7720, Val Loss: 1.6974
Epoch: 71, Loss: 0.1435
           Train Acc: 1.0000, Train Loss: 0.0000, Val Acc: 0.7680, Val Loss: 1.7345
Epoch: 81, Loss: 0.0980
           Train Acc: 1.0000, Train Loss: 0.0000, Val Acc: 0.7660, Val Loss: 1.8183


Training Epochs: 100%|██████████| 100/100 [00:00<00:00, 185.99it/s]

Epoch: 91, Loss: 0.1030
           Train Acc: 1.0000, Train Loss: 0.0000, Val Acc: 0.7640, Val Loss: 1.8376
Epoch: 100, Loss: 0.1132
           Train Acc: 1.0000, Train Loss: 0.0000, Val Acc: 0.7600, Val Loss: 1.8535
